# ETL (Extract, Transform, Load)


In [1]:
import pandas as pd
import numpy as np

I created `olympic_olap` database to create dimension table and fact tables here. 

In [2]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Parameters to connect to the default PostgreSQL database
params = {
    'dbname': 'postgres',
    'user': 'postgres',  
    'password': 'postgres', 
    'host': 'pgdb'
}

try:
    # Connect to the PostgreSQL server
    conn = psycopg2.connect(**params)
    
    # Enable autocommit so operations like creating a database are committed without having to call conn.commit()
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # Name of the new database
    new_db_name = 'olympic_olap'  # Replace with the name of the database you want to create
    
    # Ensure the database name is safe to use
    if not new_db_name.isidentifier():
        raise ValueError("Invalid database name.")
    
    # Create a new database using an f-string
    cursor.execute(f"CREATE DATABASE {new_db_name}")
    
    print("Database created successfully")
    
    # Close communication with the database
    cursor.close()
    conn.close()

except Exception as e:
    print(f"An error occurred: {e}")


Database created successfully


In [3]:

from psycopg2 import OperationalError
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [4]:
# Connection details
olap_db_name = "olympic_olap"
db_user = "postgres"
db_password = "postgres" 
db_host = "pgdb"
db_port = "5432"

In [5]:
olap_connection = create_connection(olap_db_name, db_user, db_password, db_host, db_port)
olap_cursor = olap_connection.cursor()

Connection to PostgreSQL DB successful


#### SQL Statement for Dimension and Fact tables

In [6]:
# Create DimLocation

olap_cursor.execute("""
CREATE TABLE DimLocation (
    country_code CHAR(3) NOT NULL PRIMARY KEY,
    country_name VARCHAR(255),
    continent VARCHAR(255)
);
""")

In [7]:
# Create DimEvent table

olap_cursor.execute("""
CREATE TABLE DimEvent (
    event_id SERIAL PRIMARY KEY,
    event_title VARCHAR(250),
    event_discipline VARCHAR(250),
    event_gender VARCHAR(250)
);
""")

In [8]:
# Create DimParticipant table

olap_cursor.execute("""
CREATE TABLE DimParticipant (
    participant_id SERIAL PRIMARY KEY,
    participant_title VARCHAR(255),
    participant_type VARCHAR(100)
);
""")

In [9]:
# Create DimAthlete table

olap_cursor.execute("""
CREATE TABLE DimAthlete (
    athlete_id SERIAL PRIMARY KEY,
    athlete_name VARCHAR(250),
    athlete_url VARCHAR(250)
);
""")

In [10]:
# Create DimYear table

olap_cursor.execute("""
CREATE TABLE DimYear (
    year INTEGER NOT NULL PRIMARY KEY
);
""")

In [11]:
# Create DimGame table

olap_cursor.execute("""
CREATE TABLE DimGame (
    game_slug VARCHAR(100) NOT NULL PRIMARY KEY,
    game_name VARCHAR(100),
    game_season VARCHAR(10),
    game_year INTEGER,
    country_code CHAR(3)
);
""")

In [12]:
# Create FactOlympicMedalsMeasures

olap_cursor.execute("""
CREATE TABLE FactOlympicMedalsMeasures (
    game_slug VARCHAR(100) REFERENCES DimGame(game_slug),
    participant_id INTEGER REFERENCES DimParticipant(participant_id),
    athlete_id INTEGER REFERENCES DimAthlete(athlete_id),
    event_id INTEGER REFERENCES DimEvent(event_id),
    country_code CHAR(3) NOT NULL REFERENCES DimLocation(country_code),
    year INTEGER NOT NULL REFERENCES DimYear(year),
    total_bronze_medals INTEGER,
    total_silver_medals INTEGER,
    total_gold_medals INTEGER,
    total_medals INTEGER
);
""")

In [13]:
# Create FactEconomicMeasure

olap_cursor.execute("""
CREATE TABLE FactEconomicMeasure (
    year INTEGER NOT NULL REFERENCES DimYear(year),
    country_code CHAR(3) NOT NULL REFERENCES DimLocation(country_code),
    poverty_count FLOAT,
    gdp_per_capita FLOAT,
    annual_gdp_growth FLOAT,
    servers_count INTEGER
);
""")

In [14]:
# Create FactHealthMeasure 

olap_cursor.execute("""
CREATE TABLE FactHealthMeasure (
    year INTEGER NOT NULL REFERENCES DimYear(year),
    country_code CHAR(3) NOT NULL REFERENCES DimLocation(country_code),
    daly_depression FLOAT,
    daly_schizophrenia FLOAT,
    daly_bipolar_disorder FLOAT,
    daly_eating_disorder FLOAT,
    daly_anxiety FLOAT,
    life_expectancy FLOAT,
    infant_mortality_rate FLOAT,
    current_health_expenditure FLOAT,
    government_health_expenditure FLOAT,
    private_health_expenditure FLOAT,
    external_health_expenditure FLOAT
);
""")

In [15]:
olap_cursor.execute("""
CREATE TABLE CombinedFactTable (
    id SERIAL PRIMARY KEY,
    game_slug VARCHAR(100),
    participant_id INTEGER,
    athlete_id INTEGER,
    event_id INTEGER,
    country_code CHAR(3),
    year INTEGER,
    total_bronze_medals INTEGER,
    total_silver_medals INTEGER,
    total_gold_medals INTEGER,
    total_medals INTEGER,
    poverty_count FLOAT,
    gdp_per_capita FLOAT,
    annual_gdp_growth FLOAT,
    servers_count INTEGER,
    daly_depression FLOAT,
    daly_schizophrenia FLOAT,
    daly_bipolar_disorder FLOAT,
    daly_eating_disorder FLOAT,
    daly_anxiety FLOAT,
    life_expectancy FLOAT,
    infant_mortality_rate FLOAT,
    current_health_expenditure FLOAT,
    government_health_expenditure FLOAT,
    private_health_expenditure FLOAT,
    external_health_expenditure FLOAT,
    FOREIGN KEY (game_slug) REFERENCES DimGame(game_slug),
    FOREIGN KEY (participant_id) REFERENCES DimParticipant(participant_id),
    FOREIGN KEY (athlete_id) REFERENCES DimAthlete(athlete_id),
    FOREIGN KEY (event_id) REFERENCES DimEvent(event_id),
    FOREIGN KEY (country_code) REFERENCES DimLocation(country_code),
    FOREIGN KEY (year) REFERENCES DimYear(year)
);
""")

In [16]:
olap_connection.commit()

### Loading data to Dimension tables

#### 1. DimYear

In [17]:
# Insert values from 1543 (life expectancy data) - data in the available record to 2028

olap_cursor.execute("""
INSERT INTO DimYear (year)
SELECT generate_series AS year
FROM generate_series(1543, 2028);
""")

olap_connection.commit()

#### 2. DimEvent
Loading data to DimensionEvent. For this I have chosen `event_title`, `discipline_title` and `event_gender` so that these things can be represented by `event_id` in the fact table. 

In [18]:
oltp_connection = create_connection('olympic_oltp', db_user, db_password, db_host, db_port)

oltp_cursor = oltp_connection.cursor()

Connection to PostgreSQL DB successful


In [19]:
oltp_cursor.execute("""
SELECT DISTINCT 
    event_title,
    discipline_title AS event_discipline,
    event_gender
FROM
    olympic_medals
ORDER BY
    event_title, event_gender;
""")

# Fetch the data from database
dim_event_data = oltp_cursor.fetchall()

# all column names
dim_event_data_columns = [desc[0] for desc in oltp_cursor.description]

# Create dataframe with corresponding column names
dim_event_df = pd.DataFrame(dim_event_data, columns = dim_event_data_columns)

dim_event_df

,event_title,event_discipline,event_gender
0,0.5-1t mixed,Sailing,Open
1,"0.5t mixed, race one",Sailing,Open
2,"0.5t mixed, race two",Sailing,Open
3,10000m men,Athletics,Men
4,10000m men,Speed skating,Men
...,...,...,...
1583,Women's Uneven Bars,Artistic Gymnastics,Women
1584,Women's Vault,Artistic Gymnastics,Women
1585,Women's Welter (64-69kg),Boxing,Women
1586,Yngling - Keelboat women,Sailing,Women


In [20]:
from sqlalchemy import create_engine

olap_connection_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{olap_db_name}"

# Create the engine
olap_engine = create_engine(olap_connection_url)


# Load the dataframe to DimEvent table
dim_event_df.to_sql("dimevent", con=olap_engine, if_exists="append", index=False)

588

#### 3. DimParticipant

In [21]:
oltp_cursor.execute("""
SELECT DISTINCT 
    participant_title,
    participant_type
FROM 
    olympic_medals;
""")

dim_participant_data = oltp_cursor.fetchall()

In [22]:
dim_participant_data_columns = [desc[0] for desc in oltp_cursor.description]
dim_participant_data_columns


['participant_title', 'participant_type']

In [23]:
dim_participant_df = pd.DataFrame(dim_participant_data, columns = dim_participant_data_columns)
dim_participant_df.head()

,participant_title,participant_type
0,Romania team,GameTeam
1,Tan-Fe-Pah,GameTeam
2,Pupilles de Neptune de Lille #1,GameTeam
3,Slovenia team,GameTeam
4,Gallia II,GameTeam


In [24]:
dim_participant_df.to_sql("dimparticipant", con=olap_engine, if_exists="append", index=False)

494

#### 4. DimAthlete

In [25]:
oltp_cursor.execute("""
SELECT DISTINCT 
    athlete_full_name as athlete_name,
    athlete_url
FROM 
    olympic_medals
WHERE 
    athlete_full_name IS NOT NULL AND athlete_url IS NOT NULL
ORDER BY athlete_name;
""")

dim_athlete_data = oltp_cursor.fetchall()

In [26]:
dim_athlete_data_columns = [desc[0] for desc in oltp_cursor.description]
dim_athlete_data_columns

['athlete_name', 'athlete_url']

In [27]:
dim_athlete_df = pd.DataFrame(dim_athlete_data, columns = dim_athlete_data_columns)
dim_athlete_df.head()

,athlete_name,athlete_url
0,Aage Ernst LARSEN,https://olympics.com/en/athletes/aage-ernst-la...
1,Aage Ingvar ERIKSEN,https://olympics.com/en/athletes/aage-ingvar-e...
2,Aagje Ada KOK,https://olympics.com/en/athletes/aagje-ada-kok
3,Aarne Eemeli REINI,https://olympics.com/en/athletes/aarne-eemeli-...
4,Aaron CHIA,https://olympics.com/en/athletes/aaron-chia


In [28]:
dim_athlete_df.to_sql("dimathlete", con=olap_engine, if_exists="append", index=False)

116

#### 5. DimLocation

This is the most extensive part of the ETL process. There are lot of names that does not match with each other. I have used standard names and new country codes instead which has been explained in the OLTP notebook. From there, the idea is to use the same standard names everywhere. So, i will replace the non-standard names with the one in the standard table. This will make the process more streamline and avoid data duplication and deletion. This analysis and comparisons have been documented here. 

https://uniwa-my.sharepoint.com/:x:/g/personal/23771397_student_uwa_edu_au/EVQc_vWogmVChmgQyyvDT0wBvB6OIdA7985UD4XX6MSuJg?e=BBcXYv

In [29]:
oltp_cursor.execute("""
SELECT * FROM countries;
""")

dim_location_data = oltp_cursor.fetchall()

In [30]:
dim_location_data_columns = [desc[0] for desc in oltp_cursor.description]
dim_location_data_columns

['country_code', 'country_name', 'continent']

In [31]:
dim_location_df = pd.DataFrame(dim_location_data, columns = dim_location_data_columns)
dim_location_df.head()

,country_code,country_name,continent
0,AFG,Afghanistan,Asia
1,ALA,Åland Islands,Europe
2,ALB,Albania,Europe
3,DZA,Algeria,Africa
4,ASM,American Samoa,Oceania


In [32]:
dim_location_df.to_sql("dimlocation", con=olap_engine, if_exists="append", index=False)

248

I played around with the country name running query like the one to see the name discrepancies. 

```sql
SELECT DISTINCT o.game_location
FROM olympic_hosts o
LEFT JOIN countries c ON o.game_location = c.country_name
WHERE c.country_name IS NULL;
```

Running above code we find that some countries are named differently in olympic_host file

"Australia, Sweden" -> "Australia" \
"Federal Republic of Germany" -> "Germany"
"Great Britain" -> United Kingdom of Great Britain and Northern Ireland \
"United States" -> United States of America \
"USSR" -> Russian Federation \
"Yugoslavia" -> Serbia

In [33]:
oltp_cursor.execute("""
CREATE TABLE olympic_hosts_backup AS
SELECT *
FROM olympic_hosts;
""")

oltp_connection.commit()

In [34]:
oltp_cursor.execute("""
UPDATE olympic_hosts
SET game_location = CASE
    WHEN game_location = 'Australia, Sweden' THEN 'Australia'
    WHEN game_location = 'Federal Republic of Germany' THEN 'Germany'
    WHEN game_location = 'Great Britain' THEN 'United Kingdom of Great Britain and Northern Ireland'
    WHEN game_location = 'United States' THEN 'United States of America'
    WHEN game_location = 'USSR' THEN 'Russian Federation'
    WHEN game_location = 'Yugoslavia' THEN 'Serbia'
    ELSE game_location
END;
""")

oltp_connection.commit()

In [35]:
oltp_cursor.execute("""
CREATE TABLE olympic_medals_backup AS
SELECT *
FROM olympic_medals;
""")

oltp_connection.commit()

| Old Code | New Code | Country Name                   | Remarks                                                 |
|----------|----------|--------------------------------|---------------------------------------------------------|
| AHO      | NLD      | Netherlands Antilles           | Dissolved in 2010                                       |
| ALG      | DZA      | Algeria                        |                                                         |
| ANZ      | None     | Australia and New Zealand      | Historical context, no single current ISO code          |
| BAH      | BHS      | Bahamas                        |                                                         |
| BAR      | BRB      | Barbados                       |                                                         |
| BER      | BMU      | Bermuda                        |                                                         |
| BOH      | CZE      | Bohemia                        | Historical region, now part of Czech Republic           |
| BOT      | BWA      | Botswana                       |                                                         |
| BUL      | BGR      | Bulgaria                       |                                                         |
| BUR      | MMR      | Burma                          | Now Myanmar                                             |
| CHI      | CHL      | Chile                          |                                                         |
| CRC      | CRI      | Costa Rica                     |                                                         |
| CRO      | HRV      | Croatia                        |                                                         |
| DEN      | DNK      | Denmark                        |                                                         |
| EUN      | None     | Unified Team                   | Represented former Soviet Union republics in 1992       |
| FIJ      | FJI      | Fiji                           |                                                         |
| FRG      | DEU      | Federal Republic of Germany    | Now Germany                                             |
| GDR      | DEU      | German Democratic Republic     | Now part of Germany                                     |
| GER      | DEU      | Germany                        |                                                         |
| GRE      | GRC      | Greece                         |                                                         |
| GRN      | GRD      | Grenada                        |                                                         |
| GUA      | GTM      | Guatemala                      |                                                         |
| HAI      | HTI      | Haiti                          |                                                         |
| INA      | IDN      | Indonesia                      |                                                         |
| IOA      | None     | Independent Olympic Athletes   | No standard ISO code                                    |
| IRI      | IRN      | Iran                           |                                                         |
| ISV      | VIR      | Virgin Islands, U.S.           |                                                         |
| KOS      | XKX      | Kosovo                         | Not universally recognized                              |
| KSA      | SAU      | Saudi Arabia                   |                                                         |
| KUW      | KWT      | Kuwait                         |                                                         |
| LAT      | LVA      | Latvia                         |                                                         |
| MAS      | MYS      | Malaysia                       |                                                         |
| MGL      | MNG      | Mongolia                       |                                                         |
| MIX      | None     | Mixed team                     | No standard ISO code                                    |
| MRI      | MUS      | Mauritius                      |                                                         |
| NED      | NLD      | Netherlands                    |                                                         |
| NGR      | NGA      | Nigeria                        |                                                         |
| NIG      | NER      | Niger                          |                                                         |
| OAR      | None     | Olympic Athletes from Russia   | No standard ISO code                                    |
| PAR      | PRY      | Paraguay                       |                                                         |
| PHI      | PHL      | Philippines                    |                                                         |
| POR      | PRT      | Portugal                       |                                                         |
| PUR      | PRI      | Puerto Rico                    |                                                         |
| ROC      | TWN      | Taiwan, Republic of China      | Commonly used ISO code is TWN for Taiwan                |
| RSA      | ZAF      | South Africa                   |                                                         |
| SAM      | WSM      | Samoa                          |                                                         |
| SCG      | SRB/MNE  | Serbia and Montenegro          | Dissolved, now Serbia SRB and Montenegro MNE    - Will chose SRB        |
| SLO      | SVN      | Slovenia                       |                                                         |
| SRI      | LKA      | Sri Lanka                      |                                                         |
| SUD      | SDN      | Sudan                          |                                                         |
| SUI      | CHE      | Switzerland                    |                                                         |
| TAN      | TZA      | Tanzania                       |                                                         |
| TCH      | CZE/SVK  | Czechoslovakia                 | Now Czech Republic CZE, and Slovakia SVK    - Will Choose CZE            |
| TGA      | TON      | Tonga                          |                                                         |
| TOG      | TGO      | Togo                           |                                                         |
| TPE      | TWN      | Chinese Taipei                 | Commonly used ISO code is TWN for Taiwan                |
| UAE      | ARE      | United Arab Emirates           |                                                         |
| UAR      | EGY      | United Arab Republic           | Dissolved, was a union between Egypt and Syria          |
| URS      | RUS      | Soviet Union                   | Dissolved, the largest successor state is Russia        |
| URU      | URY      | Uruguay                        |                                                         |
| VIE      | VNM      | Vietnam                        |                                                         |
| WIF      | None     | West Indies Federation         | Dissolved, was a political union of Caribbean islands   |
| YUG      | SRB/HRV  | Yugoslavia                     | Dissolved, successor states include Serbia SRB, Croatia HRV, etc. - Will choose SRB|
| ZAM      | ZMB      | Zambia                         |                                                         |
| ZIM      | ZWE      | Zimbabwe                       |                                                         |


I applied update to the `olympic_medals` first replacing the old code with new country code 

In [36]:
oltp_cursor.execute("""
-- Applying updates for each old code to the new code
UPDATE olympic_medals SET country_3_letter_code = CASE
    WHEN country_3_letter_code = 'AHO' THEN 'NLD'
    WHEN country_3_letter_code = 'ALG' THEN 'DZA'
    WHEN country_3_letter_code = 'BAH' THEN 'BHS'
    WHEN country_3_letter_code = 'BAR' THEN 'BRB'
    WHEN country_3_letter_code = 'BER' THEN 'BMU'
    WHEN country_3_letter_code = 'BOH' THEN 'CZE'
    WHEN country_3_letter_code = 'BOT' THEN 'BWA'
    WHEN country_3_letter_code = 'BUL' THEN 'BGR'
    WHEN country_3_letter_code = 'BUR' THEN 'MMR'
    WHEN country_3_letter_code = 'CHI' THEN 'CHL'
    WHEN country_3_letter_code = 'CRC' THEN 'CRI'
    WHEN country_3_letter_code = 'CRO' THEN 'HRV'
    WHEN country_3_letter_code = 'DEN' THEN 'DNK'
    WHEN country_3_letter_code = 'FIJ' THEN 'FJI'
    WHEN country_3_letter_code = 'FRG' THEN 'DEU'
    WHEN country_3_letter_code = 'GDR' THEN 'DEU'
    WHEN country_3_letter_code = 'GER' THEN 'DEU'
    WHEN country_3_letter_code = 'GRE' THEN 'GRC'
    WHEN country_3_letter_code = 'GRN' THEN 'GRD'
    WHEN country_3_letter_code = 'GUA' THEN 'GTM'
    WHEN country_3_letter_code = 'HAI' THEN 'HTI'
    WHEN country_3_letter_code = 'INA' THEN 'IDN'
    WHEN country_3_letter_code = 'IRI' THEN 'IRN'
    WHEN country_3_letter_code = 'ISV' THEN 'VIR'
    WHEN country_3_letter_code = 'KOS' THEN 'XKX'
    WHEN country_3_letter_code = 'KSA' THEN 'SAU'
    WHEN country_3_letter_code = 'KUW' THEN 'KWT'
    WHEN country_3_letter_code = 'LAT' THEN 'LVA'
    WHEN country_3_letter_code = 'MAS' THEN 'MYS'
    WHEN country_3_letter_code = 'MGL' THEN 'MNG'
    WHEN country_3_letter_code = 'MRI' THEN 'MUS'
    WHEN country_3_letter_code = 'NED' THEN 'NLD'
    WHEN country_3_letter_code = 'NGR' THEN 'NGA'
    WHEN country_3_letter_code = 'NIG' THEN 'NER'
    WHEN country_3_letter_code = 'PAR' THEN 'PRY'
    WHEN country_3_letter_code = 'PHI' THEN 'PHL'
    WHEN country_3_letter_code = 'POR' THEN 'PRT'
    WHEN country_3_letter_code = 'PUR' THEN 'PRI'
    WHEN country_3_letter_code = 'ROC' THEN 'TWN'
    WHEN country_3_letter_code = 'RSA' THEN 'ZAF'
    WHEN country_3_letter_code = 'SAM' THEN 'WSM'
    WHEN country_3_letter_code = 'SCG' THEN 'SRB'
    WHEN country_3_letter_code = 'SLO' THEN 'SVN'
    WHEN country_3_letter_code = 'SRI' THEN 'LKA'
    WHEN country_3_letter_code = 'SUD' THEN 'SDN'
    WHEN country_3_letter_code = 'SUI' THEN 'CHE'
    WHEN country_3_letter_code = 'TAN' THEN 'TZA'
    WHEN country_3_letter_code = 'TCH' THEN 'CZE'
    WHEN country_3_letter_code = 'TGA' THEN 'TON'
    WHEN country_3_letter_code = 'TOG' THEN 'TGO'
    WHEN country_3_letter_code = 'TPE' THEN 'TWN'
    WHEN country_3_letter_code = 'UAE' THEN 'ARE'
    WHEN country_3_letter_code = 'UAR' THEN 'EGY'
    WHEN country_3_letter_code = 'URS' THEN 'RUS'
    WHEN country_3_letter_code = 'URU' THEN 'URY'
    WHEN country_3_letter_code = 'VIE' THEN 'VNM'
    WHEN country_3_letter_code = 'YUG' THEN 'SRB'
    WHEN country_3_letter_code = 'ZAM' THEN 'ZMB'
    WHEN country_3_letter_code = 'ZIM' THEN 'ZWE'
    ELSE country_3_letter_code
END;
""")

In [37]:
oltp_connection.commit()

After performing join operations to see other mismatches, i found the following which has been updated. 

In [38]:
oltp_cursor.execute("""
UPDATE olympic_medals SET country_3_letter_code = CASE
    WHEN country_3_letter_code = 'EUN' THEN 'RUS'
    WHEN country_3_letter_code = 'OAR' THEN 'RUS'
    ELSE country_3_letter_code
END
""")

oltp_connection.commit()

I decided to ignore the following codes as they are ambiguous and does not contribute much to our analysis.

In [39]:
oltp_cursor.execute("""
DELETE FROM olympic_medals
WHERE country_3_letter_code IN ('ANZ', 'IOA', 'MIX', 'WIF', 'XKX');
""")


oltp_connection.commit()


Running another SQL commands made me realized the below. 

```sql

SELECT DISTINCT o.country_code
FROM life_expectancy_data o
LEFT JOIN countries c ON o.country_code = c.country_code
WHERE c.country_code IS NULL;
```

gave following

**Action**: 


"OWID_WRL" -> world -> remove it \
"OWID_KOS" -> kosovo - not recognised -> \
"OWID_USS -> RUS"

Before performing any operation, I started creating backup first. 


**a. life expectancy**

Replaced the mismatched country code in life expectancy

In [40]:
oltp_cursor.execute("""
CREATE TABLE life_expectancy_data_backup AS
SELECT *
FROM life_expectancy_data;
""")

oltp_connection.commit()

In [41]:
oltp_cursor.execute("""
-- Delete rows with "OWID_WRL" and "OWID_KOS" codes
DELETE FROM life_expectancy_data
WHERE country_code = 'OWID_WRL' OR country_code = 'OWID_KOS';
""")

oltp_cursor.execute("""
DELETE FROM life_expectancy_data
WHERE country_code IS NULL;
""")

oltp_cursor.execute("""
UPDATE life_expectancy_data
SET country_code = 'RUS'
WHERE country_code = 'OWID_USS';
""")

oltp_connection.commit()

**b. mental health data**

Removed `OWID_WRL`  as it represented world which is not required as our analysis is based on individual countries. We can always sum up all the data for each country to find the world data.

In [42]:
oltp_cursor.execute("""
CREATE TABLE mental_health_data_backup AS
SELECT *
FROM mental_health_data;
""")

oltp_cursor.execute("""
DELETE FROM mental_health_data
WHERE country_code IS NULL OR country_code = 'OWID_WRL';
""")

oltp_connection.commit()

**c. population data**

Removed data aobut regions and only kept countries' information

In [43]:
oltp_cursor.execute("""
CREATE TABLE population_data_backup AS SELECT * FROM population_data;
""")

oltp_cursor.execute("""
DELETE FROM population_data WHERE TRIM("Population") IN (
    'Advanced economies', 
    'ASEAN-5',
    'Africa (Region)',
    'Asia and Pacific', 
    'Australia and New Zealand', 
    'Caribbean', 
    'Central America', 
    'Central Asia and the Caucasus', 
    'East Asia', 
    'Eastern Europe', 
    'Emerging and Developing Asia', 
    'Emerging and Developing Europe', 
    'Emerging market and developing economies', 
    'Euro area', 
    'Europe', 
    'European Union', 
    'Kosovo', 
    'Latin America and the Caribbean', 
    'Major advanced economies (G7)', 
    'Middle East and Central Asia', 
    'Middle East (Region)', 
    'North Africa', 
    'North America', 
    'Other advanced economies', 
    'Pacific Islands', 
    'South America', 
    'South Asia', 
    'Southeast Asia', 
    'Sub-Saharan Africa', 
    'Sub-Saharan Africa (Region)', 
    'West Bank and Gaza', 
    'Western Europe', 
    'Western Hemisphere (Region)',
    'World'
);
""")

Some names had trailing whitespaces. So, used TRIM function to avoid such issues during name matching

In [44]:
oltp_cursor.execute("""
UPDATE population_data 
SET "Population" = CASE
    WHEN TRIM("Population") = 'Bahamas, The' THEN 'Bahamas'
    WHEN TRIM("Population") = 'Bolivia' THEN 'Bolivia (Plurinational State of)'
    WHEN TRIM("Population") = 'China, People''s Republic of' THEN 'China'
    WHEN TRIM("Population") = 'Congo, Dem. Rep. of the' THEN 'Democratic Republic of the Congo'
    WHEN TRIM("Population") = 'Congo, Republic of' THEN 'Congo'
    WHEN TRIM("Population") = 'Côte d''Ivoire' THEN 'Côte d’Ivoire'
    WHEN TRIM("Population") = 'Czech Republic' THEN 'Czechia'
    WHEN TRIM("Population") = 'Gambia, The' THEN 'Gambia'
    WHEN TRIM("Population") = 'Hong Kong SAR' THEN 'China, Hong Kong Special Administrative Region'
    WHEN TRIM("Population") = 'Iran' THEN 'Iran (Islamic Republic of)'
    WHEN TRIM("Population") = 'Korea, Republic of' THEN 'Republic of Korea'
    WHEN TRIM("Population") = 'Kyrgyz Republic' THEN 'Kyrgyzstan'
    WHEN TRIM("Population") = 'Lao P.D.R.' THEN 'Lao People''s Democratic Republic'
    WHEN TRIM("Population") = 'Macao SAR' THEN 'China, Macao Special Administrative Region'
    WHEN TRIM("Population") = 'Micronesia, Fed. States of' THEN 'Micronesia (Federated States of)'
    WHEN TRIM("Population") = 'Moldova' THEN 'Republic of Moldova'
    WHEN TRIM("Population") = 'North Macedonia' THEN 'North Macedonia'
    WHEN TRIM("Population") = 'São Tomé and Príncipe' THEN 'Sao Tome and Principe'
    WHEN TRIM("Population") = 'Slovak Republic' THEN 'Slovakia'
    WHEN TRIM("Population") = 'South Sudan, Republic of' THEN 'South Sudan'
    WHEN TRIM("Population") = 'Syria' THEN 'Syrian Arab Republic'
    WHEN TRIM("Population") = 'Taiwan Province of China' THEN 'Taiwan, Province of China'
    WHEN TRIM("Population") = 'Tanzania' THEN 'United Republic of Tanzania'
    WHEN TRIM("Population") = 'Türkiye, Republic of' THEN 'Turkey'
    WHEN TRIM("Population") = 'United Kingdom' THEN 'United Kingdom of Great Britain and Northern Ireland'
    WHEN TRIM("Population") = 'United States' THEN 'United States of America'
    WHEN TRIM("Population") = 'Venezuela' THEN 'Venezuela (Bolivarian Republic of)'
    WHEN TRIM("Population") = 'Vietnam' THEN 'Viet Nam'
    ELSE "Population"
END;
""")

oltp_connection.commit()

**d. economic data**

In [45]:
oltp_cursor.execute("""
-- Creating a backup of the economic_data table
CREATE TABLE economic_data_backup AS
SELECT *
FROM economic_data;
""")

For consistency, we will map the old country to new code


In [46]:
oltp_cursor.execute("""
UPDATE economic_data
SET country_code = CASE
    WHEN country_code = 'AHO' THEN 'NLD'
    WHEN country_code = 'ALG' THEN 'DZA'
    WHEN country_code = 'BAH' THEN 'BHS'
    WHEN country_code = 'BAR' THEN 'BRB'
    WHEN country_code = 'BER' THEN 'BMU'
    WHEN country_code = 'BOH' THEN 'CZE'
    WHEN country_code = 'BOT' THEN 'BWA'
    WHEN country_code = 'BUL' THEN 'BGR'
    WHEN country_code = 'BUR' THEN 'MMR'
    WHEN country_code = 'CHI' THEN 'CHL'
    WHEN country_code = 'CRC' THEN 'CRI'
    WHEN country_code = 'CRO' THEN 'HRV'
    WHEN country_code = 'DEN' THEN 'DNK'
    WHEN country_code = 'FIJ' THEN 'FJI'
    WHEN country_code = 'FRG' THEN 'DEU'
    WHEN country_code = 'GDR' THEN 'DEU'
    WHEN country_code = 'GER' THEN 'DEU'
    WHEN country_code = 'GRE' THEN 'GRC'
    WHEN country_code = 'GRN' THEN 'GRD'
    WHEN country_code = 'GUA' THEN 'GTM'
    WHEN country_code = 'HAI' THEN 'HTI'
    WHEN country_code = 'INA' THEN 'IDN'
    WHEN country_code = 'IRI' THEN 'IRN'
    WHEN country_code = 'ISV' THEN 'VIR'
    WHEN country_code = 'KOS' THEN 'XKX'
    WHEN country_code = 'KSA' THEN 'SAU'
    WHEN country_code = 'KUW' THEN 'KWT'
    WHEN country_code = 'LAT' THEN 'LVA'
    WHEN country_code = 'MAS' THEN 'MYS'
    WHEN country_code = 'MGL' THEN 'MNG'
    WHEN country_code = 'MRI' THEN 'MUS'
    WHEN country_code = 'NED' THEN 'NLD'
    WHEN country_code = 'NGR' THEN 'NGA'
    WHEN country_code = 'NIG' THEN 'NER'
    WHEN country_code = 'PAR' THEN 'PRY'
    WHEN country_code = 'PHI' THEN 'PHL'
    WHEN country_code = 'POR' THEN 'PRT'
    WHEN country_code = 'PUR' THEN 'PRI'
    WHEN country_code = 'ROC' THEN 'TWN'
    WHEN country_code = 'RSA' THEN 'ZAF'
    WHEN country_code = 'SAM' THEN 'WSM'
    WHEN country_code = 'SCG' THEN 'SRB'
    WHEN country_code = 'SLO' THEN 'SVN'
    WHEN country_code = 'SRI' THEN 'LKA'
    WHEN country_code = 'SUD' THEN 'SDN'
    WHEN country_code = 'SUI' THEN 'CHE'
    WHEN country_code = 'TAN' THEN 'TZA'
    WHEN country_code = 'TCH' THEN 'CZE'
    WHEN country_code = 'TGA' THEN 'TON'
    WHEN country_code = 'TOG' THEN 'TGO'
    WHEN country_code = 'TPE' THEN 'TWN'
    WHEN country_code = 'UAE' THEN 'ARE'
    WHEN country_code = 'UAR' THEN 'EGY'
    WHEN country_code = 'URS' THEN 'RUS'
    WHEN country_code = 'URU' THEN 'URY'
    WHEN country_code = 'VIE' THEN 'VNM'
    WHEN country_code = 'YUG' THEN 'SRB'
    WHEN country_code = 'ZAM' THEN 'ZMB'
    WHEN country_code = 'ZIM' THEN 'ZWE'
    ELSE country_code
END;
""")

```sql
SELECT DISTINCT e.country_code
FROM economic_data e
LEFT JOIN countries c ON c.country_code = e.country_code
WHERE c.country_code IS NULL;
```

Following country_code does not appear in our standard countries list. So, we will delete all the rows with the following country code.

| country_code |
|--------------|
| PST          |
| TEC          |
| TLA          |
| UMC          |
| OED          |
| ARB          |
| HIC          |
| LMC          |
| AFW          |
| MNA          |
| MEA          |
| IBT          |
| CEB          |
| MIC          |
| PRE          |
| LAC          |
| EUU          |
| SSA          |
| INX          |
| OSS          |
| LIC          |
| SSF          |
| TMN          |
| HPC          |
| TEA          |
| ECA          |
| EMU          |
| IBD          |
| LCN          |
| PSS          |
| LMY          |
| LDC          |
| NAC          |
| TSA          |
| WLD          |
| LTE          |
| SAS          |
| IDA          |
| ECS          |
| CSS          |
| AFE          |
| IDB          |
| SST          |
| FCS          |
| EAR          |
| EAS          |
| TSS          |
| EAP          |
| IDX          |

This table contains a single column titled `country_code`, with each row representing a different code from your list. This format is useful for storing or displaying a simple set of data. If you have specific attributes or additional information for each code that should be included in the table, please let me know!

In [47]:
oltp_cursor.execute("""
DELETE FROM economic_data
WHERE country_code IN (
    'PST', 'TEC', 'TLA', 'UMC', 'OED', 'ARB', 'HIC', 'LMC', 'AFW', 'MNA',
    'MEA', 'IBT', 'CEB', 'MIC', 'PRE', 'LAC', 'EUU', 'SSA', 'INX', 'OSS',
    'LIC', 'SSF', 'TMN', 'HPC', 'TEA', 'ECA', 'EMU', 'IBD', 'LCN', 'PSS',
    'LMY', 'LDC', 'NAC', 'TSA', 'WLD', 'LTE', 'SAS', 'IDA', 'ECS', 'CSS',
    'AFE', 'IDB', 'SST', 'FCS', 'EAR', 'EAS', 'TSS', 'EAP', 'IDX', 'XKX'
);
""")

In [48]:
oltp_connection.commit()

#### 6. DimGame

Used country code instead of country name on Hosts information which can be helpful during fact table creation and also during cube creation

In [49]:
oltp_cursor.execute("""
SELECT 
    o.game_slug,
    o.game_name,
    o.game_season,
    o.game_year,
    c.country_code
FROM 
    olympic_hosts o
JOIN 
    countries c ON c.country_name = o.game_location
WHERE 
    c.country_code IS NOT NULL;
""")

dim_game_data = oltp_cursor.fetchall()

In [50]:
dim_game_data_columns = [desc[0] for desc in oltp_cursor.description]
dim_game_data_columns

['game_slug', 'game_name', 'game_season', 'game_year', 'country_code']

In [51]:
dim_game_df =  pd.DataFrame(dim_game_data, columns = dim_game_data_columns)

dim_game_df.head()

,game_slug,game_name,game_season,game_year,country_code
0,beijing-2022,Beijing 2022,Winter,2022,CHN
1,tokyo-2020,Tokyo 2020,Summer,2020,JPN
2,pyeongchang-2018,PyeongChang 2018,Winter,2018,KOR
3,rio-2016,Rio 2016,Summer,2016,BRA
4,sochi-2014,Sochi 2014,Winter,2014,RUS


In [52]:
dim_game_df.to_sql("dimgame", con=olap_engine, if_exists="append", index=False)

53

### Intermediary Table For Fact Tables

Selecting certain columns from the original table to populate fact table.
I decided to create intermediary table to assist in the creation of fact tables. Since most of the things related to the olympic medals are in olympic_medals table, i selected the only thing that might be interesting to us. 

In [53]:
oltp_cursor.execute("""
SELECT
    discipline_title,
    slug_game,
    event_title,
    event_gender,
    medal_type,
    participant_type,
    participant_title,
    athlete_full_name,
    country_3_letter_code as country_code
FROM
    olympic_medals;
""")

In [54]:
olap_olympic_data = oltp_cursor.fetchall()

In [55]:
olap_olympic_data_columns = [desc[0] for desc in oltp_cursor.description]
olap_olympic_data_columns

['discipline_title',
 'slug_game',
 'event_title',
 'event_gender',
 'medal_type',
 'participant_type',
 'participant_title',
 'athlete_full_name',
 'country_code']

In [56]:
olap_olympic_df =  pd.DataFrame(olap_olympic_data, columns = olap_olympic_data_columns)

In [57]:
olap_olympic_df.head()

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_full_name,country_code
0,Football,tokyo-2020,Women,Women,GOLD,GameTeam,Canada,None,CAN
1,Archery,tokyo-2020,Women's Team,Women,SILVER,GameTeam,ROC,None,TWN
2,Softball,beijing-2008,softball women,Women,GOLD,GameTeam,Japan team,None,JPN
3,Football,seoul-1988,football men,Men,GOLD,GameTeam,Soviet Union team,None,RUS
4,Equestrian Jumping,seoul-1988,team mixed,Open,BRONZE,GameTeam,France team,None,FRA


In [58]:
olap_cursor.execute("""
CREATE TABLE olap_olympic_medals (
    discipline_title TEXT,
    slug_game TEXT,
    event_title TEXT,
    event_gender VARCHAR(250),
    medal_type TEXT,
    participant_type TEXT,
    participant_title TEXT,
    athlete_full_name TEXT,
    country_code CHAR(3)
);
""")


In [59]:
olap_connection.commit()

In [60]:
olap_olympic_df.to_sql("olap_olympic_medals", con=olap_engine, if_exists="append", index=False)

644

#### Adding participant_id to replace type and title


**Step 1: Modify the Table Structure**
First, alter `olap_olympic_medals` table to add the `participant_id` column.

```sql
ALTER TABLE olap_olympic_medals
ADD COLUMN participant_id INTEGER;
```

**Step 2: Update the `participant_id` Column**
Use an `UPDATE` statement with a `JOIN` to populate the new `participant_id` based on `participant_title` and `participant_type`. 

```sql
UPDATE olap_olympic_medals
SET participant_id = p.participant_id
FROM DimParticipant p
WHERE olap_olympic_medals.participant_title = p.participant_title
AND olap_olympic_medals.participant_type = p.participant_type;
```

**Step 3: Remove Old Columns**
After successfully updating the `participant_id` column, remove the old columns (`participant_title` and `participant_type`) 

```sql
ALTER TABLE olap_olympic_medals
DROP COLUMN participant_title,
DROP COLUMN participant_type;
```


In [61]:
olap_cursor.execute("""
ALTER TABLE olap_olympic_medals
ADD COLUMN participant_id INTEGER;
""")

olap_cursor.execute("""
UPDATE olap_olympic_medals
SET participant_id = p.participant_id
FROM DimParticipant p
WHERE olap_olympic_medals.participant_title = p.participant_title
AND olap_olympic_medals.participant_type = p.participant_type;
""")

olap_cursor.execute("""
ALTER TABLE olap_olympic_medals
DROP COLUMN participant_title,
DROP COLUMN participant_type;
""")

olap_connection.commit()

#### Adding `athelte_id` to remove information about athlete in the original data

**Step 1: Add a New Column for Athlete ID**
First, alter `olap_olympic_medals` table to add the `athlete_id` column.

```sql
ALTER TABLE olap_olympic_medals
ADD COLUMN athlete_id INTEGER;
```

**Step 2: Populate the Athlete ID Column**
Update the `athlete_id` in `olap_olympic_medals` by joining it with the `DimAthlete` table based on the `athlete_full_name`. 

```sql
UPDATE olap_olympic_medals o
SET athlete_id = a.athlete_id
FROM DimAthlete a
WHERE o.athlete_full_name = a.athlete_name;
```

In [62]:
olap_cursor.execute("""
ALTER TABLE olap_olympic_medals
ADD COLUMN athlete_id INTEGER;
""")

olap_cursor.execute("""
UPDATE olap_olympic_medals o
SET athlete_id = a.athlete_id
FROM DimAthlete a
WHERE o.athlete_full_name = a.athlete_name;
""")

olap_cursor.execute("""
ALTER TABLE olap_olympic_medals
DROP COLUMN athlete_full_name;
""")

olap_connection.commit()

#### Adding `event_id` to replace event information

**Step 1: Add New Column for Event ID**
Alter  `olap_olympic_medals` table to add the `event_id` column.

```sql
ALTER TABLE olap_olympic_medals
ADD COLUMN event_id INTEGER;
```

**Step 2: Populate the Event ID Column**
update the `event_id` in `olap_olympic_medals` by performing a join with the `DimEvent` table. The join condition will match `event_title`, `event_discipline`, and `event_gender` between the two tables.

```sql
UPDATE olap_olympic_medals o
SET event_id = e.event_id
FROM DimEvent e
WHERE o.event_title = e.event_title
AND o.event_discipline = e.event_discipline
AND o.event_gender = e.event_gender;
```

**Step 3: Remove Old Columns**
Once the `event_id` is populated, remove the `event_title`, `event_discipline`, and `event_gender` columns from the `olap_olympic_medals` table.

```sql
ALTER TABLE olap_olympic_medals
DROP COLUMN event_title,
DROP COLUMN event_discipline,
DROP COLUMN event_gender;
```

In [63]:
olap_cursor.execute("""
ALTER TABLE olap_olympic_medals
ADD COLUMN event_id INTEGER;
""")

olap_cursor.execute("""
UPDATE olap_olympic_medals o
SET event_id = e.event_id
FROM DimEvent e
WHERE o.event_title = e.event_title
AND o.discipline_title = e.event_discipline
AND o.event_gender = e.event_gender;
""")

olap_cursor.execute("""
ALTER TABLE olap_olympic_medals
DROP COLUMN event_title,
DROP COLUMN discipline_title,
DROP COLUMN event_gender;
""")

olap_connection.commit()

#### Adding year column so that it can help in querying later on

```sql

-- Adding a new column for year
ALTER TABLE olap_olympic_medals
ADD COLUMN year INTEGER;

-- Updating the year column based on the game_slug match in DimGame
UPDATE olap_olympic_medals o
SET year = g.game_year
FROM DimGame g
WHERE o.game_slug = g.game_slug;

```

In [64]:
olap_cursor.execute("""
ALTER TABLE olap_olympic_medals
ADD COLUMN year INTEGER;
""")

In [65]:
olap_cursor.execute("""
UPDATE olap_olympic_medals o
SET year = g.game_year
FROM DimGame g
WHERE o.slug_game = g.game_slug;
""")

### Adding number of medals won by the country in different year

```sql
SELECT
    country_code,
    year AS yr,
    COUNT(CASE WHEN medal_type = 'BRONZE' THEN 1 END) AS total_bronze_medals,
    COUNT(CASE WHEN medal_type = 'SILVER' THEN 1 END) AS total_silver_medals,
    COUNT(CASE WHEN medal_type = 'GOLD' THEN 1 END) AS total_gold_medals,
    COUNT(*) AS total_medals
FROM
    olap_olympic_medals
GROUP BY
    country_code,
    yr
ORDER BY 
    yr, total_medals DESC;
```

In [66]:
olap_cursor.execute("""
SELECT
    country_code,
	participant_id,
	athlete_id,
    event_id,
	slug_game as game_slug,
    year,
    COUNT(CASE WHEN medal_type = 'BRONZE' THEN 1 END) AS total_bronze_medals,
    COUNT(CASE WHEN medal_type = 'SILVER' THEN 1 END) AS total_silver_medals,
    COUNT(CASE WHEN medal_type = 'GOLD' THEN 1 END) AS total_gold_medals,
    COUNT(*) AS total_medals
FROM
    olap_olympic_medals
GROUP BY
    country_code,
	participant_id,
	athlete_id,
    event_id,
	slug_game,
    year
ORDER BY 
    year, total_medals DESC;
""")

fact_olympic_measure_data = olap_cursor.fetchall()

In [67]:
fact_olympic_measure_data_columns = [desc[0] for desc in olap_cursor.description]
fact_olympic_measure_data_columns

['country_code',
 'participant_id',
 'athlete_id',
 'event_id',
 'game_slug',
 'year',
 'total_bronze_medals',
 'total_silver_medals',
 'total_gold_medals',
 'total_medals']

In [68]:
fact_olympic_measure_df =  pd.DataFrame(fact_olympic_measure_data, columns = fact_olympic_measure_data_columns)
fact_olympic_measure_df

,country_code,participant_id,athlete_id,event_id,game_slug,year,total_bronze_medals,total_silver_medals,total_gold_medals,total_medals
0,GRC,NaN,NaN,129,athens-1896,1896,0,1,1,2
1,GRC,NaN,NaN,455,athens-1896,1896,0,1,1,2
2,GRC,NaN,NaN,611,athens-1896,1896,2,0,0,2
3,USA,NaN,NaN,694,athens-1896,1896,0,2,0,2
4,USA,NaN,NaN,873,athens-1896,1896,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...
21578,TWN,NaN,342.0,1101,beijing-2022,2022,1,0,0,1
21579,SVN,NaN,11017.0,1546,beijing-2022,2022,0,0,1,1
21580,TWN,NaN,318.0,935,beijing-2022,2022,0,0,1,1
21581,AUT,NaN,305.0,1095,beijing-2022,2022,0,0,1,1


In [69]:
fact_olympic_measure_df.to_sql("factolympicmedalsmeasures", con=olap_engine, if_exists="append", index=False)

583

In [70]:
### Economic Measure

In [71]:
oltp_cursor.execute("""
SELECT time_year as year, 
    country_code, 
    poverty_ratio as poverty_count,
    gdp_per_capita_usd as gdp_per_capita,
    gdp_per_capita_growth as annual_gdp_growth,
    secure_internet_servers_per_million as servers_count
FROM economic_data;
""")

fact_economic_measures = oltp_cursor.fetchall()

In [72]:
oltp_connection.commit()

In [73]:
fact_economic_measures_columns = [desc[0] for desc in oltp_cursor.description]
fact_economic_measures_columns

['year',
 'country_code',
 'poverty_count',
 'gdp_per_capita',
 'annual_gdp_growth',
 'servers_count']

In [74]:
fact_economic_measures_df =  pd.DataFrame(fact_economic_measures, columns = fact_economic_measures_columns)
fact_economic_measures_df

,year,country_code,poverty_count,gdp_per_capita,annual_gdp_growth,servers_count
0,1973,DEU,0.0,5046.755103,4.448017,1049.317943
1,2006,TUV,3.6,2402.480106,1.233859,92.038656
2,1960,ABW,8.2,6283.001443,16.263941,89.694143
3,1961,ABW,8.2,6283.001443,16.263941,89.694143
4,1962,ABW,8.2,6283.001443,16.263941,89.694143
...,...,...,...,...,...,...
13819,2019,ZWE,39.8,1421.868596,-8.177320,64.540886
13820,2020,ZWE,39.8,1372.696674,-9.670405,70.646050
13821,2021,ZWE,39.8,1773.920411,6.271613,70.646050
13822,2022,ZWE,39.8,1676.821489,4.387997,70.646050


In [75]:
fact_economic_measures_df.to_sql("facteconomicmeasure", con=olap_engine, if_exists="append", index=False)

824

In [76]:
oltp_cursor.execute("""
SELECT
    COALESCE(e.time_year, CAST(l.year AS VARCHAR(255)), CAST(m.year AS VARCHAR(255))) AS year,
    COALESCE(e.country_code, l.country_code, m.country_code) AS country_code,
    m.daly_depression,
    m.daly_schizophrenia,
    m.daly_bipolar_disorder,
    m.daly_eating_disorder,
    m.daly_anxiety,
    l.life_expectancy,
    e.infant_mortality_rate,
    e.health_expenditure_pct_gdp AS current_health_expenditure,
    e.gov_health_expenditure_per_capita_usd AS government_health_expenditure,
    e.private_health_expenditure_per_capita_usd AS private_health_expenditure,
    e.external_health_expenditure_per_capita_usd AS external_health_expenditure
FROM
    economic_data e
FULL OUTER JOIN
    life_expectancy_data l ON e.time_year = CAST(l.year AS VARCHAR(255)) AND e.country_code = l.country_code
FULL OUTER JOIN
    mental_health_data m ON COALESCE(e.time_year, CAST(l.year AS VARCHAR(255))) = CAST(m.year AS VARCHAR(255)) AND COALESCE(e.country_code, l.country_code) = m.country_code;
""")

In [77]:
olap_health_measure = oltp_cursor.fetchall()

In [78]:
olap_health_measure_columns = [desc[0] for desc in oltp_cursor.description]
olap_health_measure_columns

['year',
 'country_code',
 'daly_depression',
 'daly_schizophrenia',
 'daly_bipolar_disorder',
 'daly_eating_disorder',
 'daly_anxiety',
 'life_expectancy',
 'infant_mortality_rate',
 'current_health_expenditure',
 'government_health_expenditure',
 'private_health_expenditure',
 'external_health_expenditure']

In [79]:
olap_health_measure_df =  pd.DataFrame(olap_health_measure, columns = olap_health_measure_columns)
olap_health_measure_df

,year,country_code,daly_depression,daly_schizophrenia,daly_bipolar_disorder,daly_eating_disorder,daly_anxiety,life_expectancy,infant_mortality_rate,current_health_expenditure,government_health_expenditure,private_health_expenditure,external_health_expenditure
0,1543,GBR,NaN,NaN,NaN,NaN,NaN,33.94,NaN,NaN,NaN,NaN,NaN
1,1548,GBR,NaN,NaN,NaN,NaN,NaN,38.82,NaN,NaN,NaN,NaN,NaN
2,1553,GBR,NaN,NaN,NaN,NaN,NaN,39.59,NaN,NaN,NaN,NaN,NaN
3,1558,GBR,NaN,NaN,NaN,NaN,NaN,22.38,NaN,NaN,NaN,NaN,NaN
4,1563,GBR,NaN,NaN,NaN,NaN,NaN,36.66,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19399,2023,WSM,NaN,NaN,NaN,NaN,NaN,NaN,14.4,5.326137,146.576807,24.657693,31.199702
19400,2023,YEM,NaN,NaN,NaN,NaN,NaN,NaN,46.7,4.252857,6.445508,51.927447,4.927408
19401,2023,ZAF,NaN,NaN,NaN,NaN,NaN,NaN,26.4,8.583298,304.086149,178.993024,6.564406
19402,2023,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,40.2,5.617884,23.363919,7.747271,22.642471


In [80]:
olap_health_measure_interpolated = olap_health_measure_df.groupby('country_code').apply(lambda group: group.interpolate(method='linear', limit_direction='both'))

# import numpy as np

# data_numeric_interpolation = data.groupby('Country Code').apply(
#     lambda group: group.select_dtypes(include=[np.number]).interpolate(method='linear', limit_direction='both'), include_groups=True
# )
# Display the first few rows after grouped interpolation to verify changes
olap_health_measure_interpolated

/tmp/ipykernel_24110/803702965.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  olap_health_measure_interpolated = olap_health_measure_df.groupby('country_code').apply(lambda group: group.interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_24110/803702965.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  olap_health_measure_interpolated = olap_health_measure_df.groupby('country_code').apply(lambda group: group.interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_24110/803702965.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  olap_health_measure_interpolated = olap_health_me

year country_code  daly_depression  daly_schizophrenia  \
country_code                                                                 
ABW          1990   1950          ABW              NaN                 NaN   
             2225   1951          ABW              NaN                 NaN   
             2460   1952          ABW              NaN                 NaN   
             2695   1953          ABW              NaN                 NaN   
             2930   1954          ABW              NaN                 NaN   
...                  ...          ...              ...                 ...   
ZWE          18499  2019          ZWE         544.5316           128.21603   
             18735  2020          ZWE         544.5316           128.21603   
             18971  2021          ZWE         544.5316           128.21603   
             19187  2022          ZWE         544.5316           128.21603   
             19403  2023          ZWE         544.5316           128.21603   

                    daly_bipolar_disorder  daly_eating_disorder  daly_anxiety  \
country_code                                                                    
ABW          1990                     NaN                   NaN           NaN   
             2225                     NaN                   NaN           NaN   
             2460                     NaN                   NaN           NaN   
             2695                     NaN                   NaN           NaN   
             2930                     NaN                   NaN           NaN   
...                                   ...                   ...           ...   
ZWE          18499              115.38145             21.011127     298.20325   
             18735              115.38145             21.011127     298.20325   
             18971              115.38145             21.011127     298.20325   
             19187              115.38145             21.011127     298.20325   
             19403              115.38145             21.011127     298.20325   

                    life_expectancy  infant_mortality_rate  \
country_code                                                 
ABW          1990           57.1739                   38.5   
             2225           57.6605                   38.5   
             2460           58.6525                   38.5   
             2695           59.4873                   38.5   
             2930           60.4084                   38.5   
...                             ...                    ...   
ZWE          18499          61.2925                   37.1   
             18735          61.1242                   36.6   
             18971          59.2531                   35.7   
             19187          59.2531                   35.7   
             19403          59.2531                   35.7   

                    current_health_expenditure  government_health_expenditure  \
country_code                                                                    
ABW          1990                     5.228842                      64.093112   
             2225                     5.228842                      64.093112   
             2460                     5.228842                      64.093112   
             2695                     5.228842                      64.093112   
             2930                     5.228842                      64.093112   
...                                        ...                            ...   
ZWE          18499                    3.659765                       7.818598   
             18735                    3.425581                      11.195503   
             18971                    3.425581                      11.195503   
             19187                    3.425581                      11.195503   
             19403                    3.425581                      11.195503   

                    private_health_expenditure  external_health_expenditure  
country_code

In [81]:
# Calculate the median for each column and fill missing values with the median
columns_with_na = olap_health_measure_interpolated.columns[olap_health_measure_interpolated.isna().any()].tolist()  # Columns that have missing values
medians = olap_health_measure_interpolated[columns_with_na].median()

# Fill missing values with medians
olap_health_measure_interpolated_filled = olap_health_measure_interpolated.fillna(medians)

# Check the summary of missing values after filling to confirm
olap_health_measure_interpolated_filled.isna().sum()


year                             0
country_code                     0
daly_depression                  0
daly_schizophrenia               0
daly_bipolar_disorder            0
daly_eating_disorder             0
daly_anxiety                     0
life_expectancy                  0
infant_mortality_rate            0
current_health_expenditure       0
government_health_expenditure    0
private_health_expenditure       0
external_health_expenditure      0
dtype: int64

In [82]:
olap_health_measure_interpolated_filled

year country_code  daly_depression  daly_schizophrenia  \
country_code                                                                 
ABW          1990   1950          ABW        652.26294          175.044965   
             2225   1951          ABW        652.26294          175.044965   
             2460   1952          ABW        652.26294          175.044965   
             2695   1953          ABW        652.26294          175.044965   
             2930   1954          ABW        652.26294          175.044965   
...                  ...          ...              ...                 ...   
ZWE          18499  2019          ZWE        544.53160          128.216030   
             18735  2020          ZWE        544.53160          128.216030   
             18971  2021          ZWE        544.53160          128.216030   
             19187  2022          ZWE        544.53160          128.216030   
             19403  2023          ZWE        544.53160          128.216030   

                    daly_bipolar_disorder  daly_eating_disorder  daly_anxiety  \
country_code                                                                    
ABW          1990               131.12288             32.982437    379.401065   
             2225               131.12288             32.982437    379.401065   
             2460               131.12288             32.982437    379.401065   
             2695               131.12288             32.982437    379.401065   
             2930               131.12288             32.982437    379.401065   
...                                   ...                   ...           ...   
ZWE          18499              115.38145             21.011127    298.203250   
             18735              115.38145             21.011127    298.203250   
             18971              115.38145             21.011127    298.203250   
             19187              115.38145             21.011127    298.203250   
             19403              115.38145             21.011127    298.203250   

                    life_expectancy  infant_mortality_rate  \
country_code                                                 
ABW          1990           57.1739                   38.5   
             2225           57.6605                   38.5   
             2460           58.6525                   38.5   
             2695           59.4873                   38.5   
             2930           60.4084                   38.5   
...                             ...                    ...   
ZWE          18499          61.2925                   37.1   
             18735          61.1242                   36.6   
             18971          59.2531                   35.7   
             19187          59.2531                   35.7   
             19403          59.2531                   35.7   

                    current_health_expenditure  government_health_expenditure  \
country_code                                                                    
ABW          1990                     5.228842                      64.093112   
             2225                     5.228842                      64.093112   
             2460                     5.228842                      64.093112   
             2695                     5.228842                      64.093112   
             2930                     5.228842                      64.093112   
...                                        ...                            ...   
ZWE          18499                    3.659765                       7.818598   
             18735                    3.425581                      11.195503   
             18971                    3.425581                      11.195503   
             19187                    3.425581                      11.195503   
             19403                    3.425581                      11.195503   

                    private_health_expenditure  external_health_expenditure  
country_code

In [83]:
olap_health_measure_interpolated_filled.to_sql("facthealthmeasure", con=olap_engine, if_exists="append", index=False)

404

In [84]:
olap_cursor.execute("""
SELECT 
    olympics.game_slug,
    olympics.participant_id,
    olympics.athlete_id,
    olympics.event_id,
    COALESCE(olympics.country_code, econ.country_code, health.country_code) AS country_code,
    COALESCE(olympics.year, econ.year, health.year) AS year,
    olympics.total_bronze_medals,
    olympics.total_silver_medals,
    olympics.total_gold_medals,
    olympics.total_medals,
    econ.poverty_count,
    econ.gdp_per_capita,
    econ.annual_gdp_growth,
    econ.servers_count,
    health.daly_depression,
    health.daly_schizophrenia,
    health.daly_bipolar_disorder,
    health.daly_eating_disorder,
    health.daly_anxiety,
    health.life_expectancy,
    health.infant_mortality_rate,
    health.current_health_expenditure,
    health.government_health_expenditure,
    health.private_health_expenditure,
    health.external_health_expenditure
FROM FactOlympicMedalsMeasures AS olympics
FULL OUTER JOIN FactEconomicMeasure AS econ
    ON olympics.country_code = econ.country_code AND olympics.year = econ.year
FULL OUTER JOIN FactHealthMeasure AS health
    ON COALESCE(olympics.country_code, econ.country_code) = health.country_code 
    AND COALESCE(olympics.year, econ.year) = health.year;
""")

In [85]:
combined_fact_table = olap_cursor.fetchall()

In [86]:
combine_fact_table_columns = [desc[0] for desc in olap_cursor.description]
combine_fact_table_columns

['game_slug',
 'participant_id',
 'athlete_id',
 'event_id',
 'country_code',
 'year',
 'total_bronze_medals',
 'total_silver_medals',
 'total_gold_medals',
 'total_medals',
 'poverty_count',
 'gdp_per_capita',
 'annual_gdp_growth',
 'servers_count',
 'daly_depression',
 'daly_schizophrenia',
 'daly_bipolar_disorder',
 'daly_eating_disorder',
 'daly_anxiety',
 'life_expectancy',
 'infant_mortality_rate',
 'current_health_expenditure',
 'government_health_expenditure',
 'private_health_expenditure',
 'external_health_expenditure']

In [87]:
combined_fact_table_df =  pd.DataFrame(combined_fact_table, columns = combine_fact_table_columns)
combined_fact_table_df

,game_slug,participant_id,athlete_id,event_id,country_code,year,total_bronze_medals,total_silver_medals,total_gold_medals,total_medals,...,daly_schizophrenia,daly_bipolar_disorder,daly_eating_disorder,daly_anxiety,life_expectancy,infant_mortality_rate,current_health_expenditure,government_health_expenditure,private_health_expenditure,external_health_expenditure
0,None,NaN,NaN,NaN,ABW,1950,NaN,NaN,NaN,NaN,...,175.044965,131.12288,32.982437,379.401065,57.1739,38.5,5.228842,64.093112,50.730973,1.112028
1,None,NaN,NaN,NaN,ABW,1951,NaN,NaN,NaN,NaN,...,175.044965,131.12288,32.982437,379.401065,57.6605,38.5,5.228842,64.093112,50.730973,1.112028
2,None,NaN,NaN,NaN,ABW,1952,NaN,NaN,NaN,NaN,...,175.044965,131.12288,32.982437,379.401065,58.6525,38.5,5.228842,64.093112,50.730973,1.112028
3,None,NaN,NaN,NaN,ABW,1953,NaN,NaN,NaN,NaN,...,175.044965,131.12288,32.982437,379.401065,59.4873,38.5,5.228842,64.093112,50.730973,1.112028
4,None,NaN,NaN,NaN,ABW,1954,NaN,NaN,NaN,NaN,...,175.044965,131.12288,32.982437,379.401065,60.4084,38.5,5.228842,64.093112,50.730973,1.112028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39693,None,NaN,NaN,NaN,ZWE,2019,NaN,NaN,NaN,NaN,...,128.216030,115.38145,21.011127,298.203250,61.2925,37.1,3.659765,7.818598,13.044234,33.945674
39694,None,NaN,NaN,NaN,ZWE,2020,NaN,NaN,NaN,NaN,...,128.216030,115.38145,21.011127,298.203250,61.1242,36.6,3.425581,11.195503,11.267075,28.213933
39695,None,NaN,NaN,NaN,ZWE,2021,NaN,NaN,NaN,NaN,...,128.216030,115.38145,21.011127,298.203250,59.2531,35.7,3.425581,11.195503,11.267075,28.213933
39696,None,NaN,NaN,NaN,ZWE,2022,NaN,NaN,NaN,NaN,...,128.216030,115.38145,21.011127,298.203250,59.2531,35.7,3.425581,11.195503,11.267075,28.213933


In [88]:
combined_fact_table_df.to_sql("combinedfacttable", con=olap_engine, if_exists="append", index=False)

698